# Tutorial 2: Inferencia bayesiana y decisiones con estado oculto continuo.

---
# Configuración

## Instalar e importar gadget de comentarios

In [1]:
# @title Install and import feedback gadget

!pip3 install vibecheck datatops --quiet

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_cn",
            "user_key": "y1x3mpx5",
        },
    ).render()


feedback_prefix = "W3D1_T2"

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00


In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import gamma as gamma_distribution
from matplotlib.transforms import Affine2D

## Configuración de figura

In [3]:
# @title Figure Settings
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import FloatSlider, Dropdown, interactive_output
from ipywidgets import interact, fixed, HBox, Layout, VBox, interactive, Label
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

import warnings
warnings.filterwarnings("ignore")

## Funciones de graficado

In [4]:
# @title Plotting Functions

def plot_mixture_prior(x, gaussian1, gaussian2, combined):
  """ Plots a prior made of a mixture of gaussians

  Args:
    x (numpy array of floats):         points at which the likelihood has been evaluated
    gaussian1 (numpy array of floats): normalized probabilities for Gaussian 1 evaluated at each `x`
    gaussian2 (numpy array of floats): normalized probabilities for Gaussian 2 evaluated at each `x`
    posterior (numpy array of floats): normalized probabilities for the posterior evaluated at each `x`

  Returns:
    Nothing
  """
  fig, ax = plt.subplots()
  ax.plot(x, gaussian1, '--b', LineWidth=2, label='Gaussian 1')
  ax.plot(x, gaussian2, '-.b', LineWidth=2, label='Gaussian 2')
  ax.plot(x, combined, '-r', LineWidth=2, label='Gaussian Mixture')
  ax.legend()
  ax.set_ylabel('Probability')
  ax.set_xlabel('Orientation (Degrees)')


def plot_gaussian(mu, sigma):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  y = gaussian(x, mu, sigma)

  plt.figure(figsize=(6, 4))
  plt.plot(x, y, c='blue')
  plt.fill_between(x, y, color='b', alpha=0.2)
  plt.ylabel('$\mathcal{N}(x, \mu, \sigma^2)$')
  plt.xlabel('x')
  plt.yticks([])
  plt.show()


def plot_losses(mu, sigma):
  x = np.linspace(-2, 2, 400, endpoint=True)
  y = gaussian(x, mu, sigma)
  error = x - mu

  mse_loss = (error)**2
  abs_loss = np.abs(error)
  zero_one_loss = (np.abs(error) >= 0.02).astype(np.float)

  fig, (ax_gaus, ax_error) = plt.subplots(2, 1, figsize=(6, 8))
  ax_gaus.plot(x, y, color='blue', label='true distribution')
  ax_gaus.fill_between(x, y, color='blue', alpha=0.2)
  ax_gaus.set_ylabel('$\\mathcal{N}(x, \\mu, \\sigma^2)$')
  ax_gaus.set_xlabel('x')
  ax_gaus.set_yticks([])
  ax_gaus.legend(loc='upper right')

  ax_error.plot(x, mse_loss, color='c', label='Mean Squared Error', linewidth=3)
  ax_error.plot(x, abs_loss, color='m', label='Absolute Error', linewidth=3)
  ax_error.plot(x, zero_one_loss, color='y', label='Zero-One Loss', linewidth=3)
  ax_error.legend(loc='upper right')
  ax_error.set_xlabel('$\\hat{\\mu}$')
  ax_error.set_ylabel('Error')
  plt.show()


def gaussian_mixture(mu1, mu2, sigma1, sigma2, factor):
  assert 0.0 < factor < 1.0
  x = np.linspace(-7.0, 7.0, 1000, endpoint=True)
  y_1 = gaussian(x, mu1, sigma1)
  y_2 = gaussian(x, mu2, sigma2)
  mixture = y_1 * factor + y_2 * (1.0 - factor)

  plt.figure(figsize=(8, 6))
  plt.plot(x, y_1, c='deepskyblue', label='p(x)', linewidth=3.0)
  plt.fill_between(x, y_1, color='deepskyblue', alpha=0.2)
  plt.plot(x, y_2, c='aquamarine', label='q(x)', linewidth=3.0)
  plt.fill_between(x, y_2, color='aquamarine', alpha=0.2)
  plt.plot(x, mixture, c='b', label='$\pi \cdot p(x) + (1-\pi) \cdot q(x)$',  linewidth=3.0)
  plt.fill_between(x, mixture, color='b', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.xlabel('x')
  plt.show()


def plot_utility_mixture_dist(mu1, sigma1, mu2, sigma2, mu_g, sigma_g,
                              mu_loc, mu_dist, plot_utility_row=True):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)
  mu_post, sigma_post = product_guassian(mu1, mu2, sigma1, sigma2)
  posterior = gaussian(x, mu_post, sigma_post)
  gain = gaussian(x, mu_g, sigma_g)/2

  sigma_mix, factor = 1.0, 0.5
  mu_mix1, mu_mix2 = mu_loc - mu_dist/2, mu_loc + mu_dist/2
  gaus_mix1, gaus_mix2 = gaussian(x, mu_mix1, sigma_mix), gaussian(x, mu_mix2, sigma_mix)
  loss = factor * gaus_mix1 + (1 - factor) * gaus_mix2
  utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)

  if plot_utility_row:
    plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility)
  else:
    plot_bayes_row(x, prior, likelihood, posterior)

  return None


def plot_mvn2d(mu1, mu2, sigma1, sigma2, corr):
  x, y = np.mgrid[-2:2:.02, -2:2:.02]
  cov12 = corr * sigma1 * sigma2
  z = mvn2d(x, y, mu1, mu2, sigma1, sigma2, cov12)

  plt.figure(figsize=(6, 6))
  plt.contourf(x, y, z, cmap='Reds')
  plt.axis("off")
  plt.show()


def plot_marginal(sigma1, sigma2, c_x, c_y, corr):
  mu1, mu2 = 0.0, 0.0
  cov12 = corr * sigma1 * sigma2
  xx, yy = np.mgrid[-2:2:.02, -2:2:.02]
  x, y = xx[:, 0], yy[0]
  p_x = gaussian(x, mu1, sigma1)
  p_y = gaussian(y, mu2, sigma2)
  zz = mvn2d(xx, yy, mu1, mu2, sigma1, sigma2, cov12)

  mu_x_y = mu1+cov12*(c_y-mu2)/sigma2**2
  mu_y_x = mu2+cov12*(c_x-mu1)/sigma1**2
  sigma_x_y = np.sqrt(sigma1**2 - cov12**2/sigma2**2)
  sigma_y_x = np.sqrt(sigma2**2 - cov12**2/sigma1**2)
  p_x_y = gaussian(x, mu_x_y, sigma_x_y)
  p_y_x = gaussian(x, mu_y_x, sigma_y_x)

  p_c_y = gaussian(mu_x_y-sigma_x_y, mu_x_y, sigma_x_y)
  p_c_x = gaussian(mu_y_x-sigma_y_x, mu_y_x, sigma_y_x)

  # definitions for the axes
  left, width = 0.1, 0.65
  bottom, height = 0.1, 0.65
  spacing = 0.01

  rect_z = [left, bottom, width, height]
  rect_x = [left, bottom + height + spacing, width, 0.2]
  rect_y = [left + width + spacing, bottom, 0.2, height]

  # start with a square Figure
  fig = plt.figure(figsize=(8, 8))

  ax_z = fig.add_axes(rect_z)
  ax_x = fig.add_axes(rect_x, sharex=ax_z)
  ax_y = fig.add_axes(rect_y, sharey=ax_z)

  ax_z.set_axis_off()
  ax_x.set_axis_off()
  ax_y.set_axis_off()
  ax_x.set_xlim(np.min(x), np.max(x))
  ax_y.set_ylim(np.min(y), np.max(y))

  ax_z.contourf(xx, yy, zz, cmap='Greys')
  ax_z.hlines(c_y, mu_x_y-sigma_x_y, mu_x_y+sigma_x_y, color='c', zorder=9, linewidth=3)
  ax_z.vlines(c_x, mu_y_x-sigma_y_x, mu_y_x+sigma_y_x, color='m', zorder=9, linewidth=3)

  ax_x.plot(x, p_x, label='$p(x)$', c = 'b', linewidth=3)
  ax_x.plot(x, p_x_y, label='$p(x|y = C_y)$', c = 'c', linestyle='dashed', linewidth=3)
  ax_x.hlines(p_c_y, mu_x_y-sigma_x_y, mu_x_y+sigma_x_y, color='c', linestyle='dashed', linewidth=3)

  ax_y.plot(p_y, y, label='$p(y)$', c = 'r', linewidth=3)
  ax_y.plot(p_y_x, y, label='$p(y|x = C_x)$', c = 'm', linestyle='dashed', linewidth=3)
  ax_y.vlines(p_c_x, mu_y_x-sigma_y_x, mu_y_x+sigma_y_x, color='m', linestyle='dashed', linewidth=3)

  ax_x.legend(loc="upper left", frameon=False)
  ax_y.legend(loc="lower right", frameon=False)

  plt.show()


def plot_bayes(mu1, mu2, sigma1, sigma2):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)

  mu_post, sigma_post = product_guassian(mu1, mu2, sigma1, sigma2)
  posterior = gaussian(x, mu_post, sigma_post)

  plt.figure(figsize=(8, 6))
  plt.plot(x, prior, c='b', label='prior')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='likelihood')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.plot(x, posterior, c='k', label='posterior')
  plt.fill_between(x, posterior, color='k', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.ylabel('$\mathcal{N}(x, \mu, \sigma^2)$')
  plt.xlabel('x')
  plt.show()


def plot_information(mu1, sigma1, mu2, sigma2):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  mu3, sigma3 = product_guassian(mu1, mu2, sigma1, sigma2)
  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)
  posterior = gaussian(x, mu3, sigma3)

  plt.figure(figsize=(8, 6))
  plt.plot(x, prior, c='b', label='Satellite')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='Space Mouse')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.plot(x, posterior, c='k', label='Center')
  plt.fill_between(x, posterior, color='k', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.ylabel('$\mathcal{N}(x, \mu, \sigma^2)$')
  plt.xlabel('x')
  plt.show()


def plot_information_global(mu3, sigma3, mu1, mu2):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  sigma1, sigma2 = reverse_product(mu3, sigma3, mu1, mu2)
  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)
  posterior = gaussian(x, mu3, sigma3)

  plt.figure(figsize=(8, 6))
  plt.plot(x, prior, c='b', label='Satellite')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='Space Mouse')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.plot(x, posterior, c='k', label='Center')
  plt.fill_between(x, posterior, color='k', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.ylabel('$\mathcal{N}(x, \mu, \sigma^2)$')
  plt.xlabel('x')
  plt.show()


def plot_loss_utility_gaussian(loss_f, mu, sigma, mu_true):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  posterior = gaussian(x, mu, sigma)
  y_label = "$p(x)$"

  plot_loss_utility(x, posterior, loss_f, mu_true, y_label)


def plot_loss_utility_mixture(loss_f, mu1, mu2, sigma1, sigma2, factor, mu_true):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  y_1 = gaussian(x, mu1, sigma1)
  y_2 = gaussian(x, mu2, sigma2)
  posterior = y_1 * factor + y_2 * (1.0 - factor)
  y_label = "$\pi \cdot p(x) + (1-\pi) \cdot q(x)$"
  plot_loss_utility(x, posterior, loss_f, mu_true, y_label)


def plot_loss_utility(x, posterior, loss_f, mu_true, y_label):
  mean, median, mode = calc_mean_mode_median(x, posterior)

  loss = calc_loss_func(loss_f, mu_true, x)

  utility = calc_expected_loss(loss_f, posterior, x)
  min_expected_loss = x[np.argmin(utility)]

  plt.figure(figsize=(12, 8))
  plt.subplot(2, 2, 1)
  plt.title("Probability")
  plt.plot(x, posterior, c='b')
  plt.fill_between(x, posterior, color='b', alpha=0.2)
  plt.yticks([])
  plt.xlabel('x')
  plt.ylabel(y_label)
  plt.axvline(mean, ls='dashed', color='red', label='Mean')
  plt.axvline(median, ls='dashdot', color='blue', label='Median')
  plt.axvline(mode, ls='dotted', color='green', label='Mode')
  plt.legend(loc="upper left")

  plt.subplot(2, 2, 2)
  plt.title(loss_f)
  plt.plot(x, loss, c='c', label=loss_f)
  plt.ylabel('loss')
  plt.xlabel('x')

  plt.subplot(2, 2, 3)
  plt.title("Expected Loss")
  plt.plot(x, utility, c='y', label='$\mathbb{E}[L]$')
  plt.axvline(min_expected_loss, ls='dashed', color='red', label='$Min~ \mathbb{E}[Loss]$')
  plt.legend(loc="lower right")
  plt.xlabel('x')
  plt.ylabel('$\mathbb{E}[L]$')

  plt.show()


def plot_loss_utility_bayes(mu1, mu2, sigma1, sigma2, mu_true, loss_f):
  x = np.linspace(-4, 4, 1000, endpoint=True)

  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)

  mu_post, sigma_post = product_guassian(mu1, mu2, sigma1, sigma2)
  posterior = gaussian(x, mu_post, sigma_post)

  loss = calc_loss_func(loss_f, mu_true, x)

  utility = - calc_expected_loss(loss_f, posterior, x)

  plt.figure(figsize=(18, 5))
  plt.subplot(1, 3, 1)

  plt.title("Posterior distribution")
  plt.plot(x, prior, c='b', label='prior')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='likelihood')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.plot(x, posterior, c='k', label='posterior')
  plt.fill_between(x, posterior, color='k', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.xlabel('x')

  plt.subplot(1, 3, 2)
  plt.title(loss_f)
  plt.plot(x, loss, c='c')
  plt.ylabel('loss')

  plt.subplot(1, 3, 3)
  plt.title("Expected utility")
  plt.plot(x, utility, c='y', label='utility')
  plt.legend(loc="upper left")

  plt.show()


def plot_simple_utility_gaussian(mu, sigma, mu_g, mu_c, sigma_g, sigma_c):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  posterior = gaussian(x, mu, sigma)
  gain = gaussian(x, mu_g, sigma_g)
  loss = gaussian(x, mu_c, sigma_c)
  utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)

  plt.figure(figsize=(15, 5))
  plt.subplot(1, 3, 1)
  plt.title("Probability")
  plt.plot(x, posterior, c='b', label='posterior')
  plt.fill_between(x, posterior, color='b', alpha=0.2)
  plt.yticks([])
  plt.xlabel('x')

  plt.subplot(1, 3, 2)
  plt.title("utility function")
  plt.plot(x, gain, c='m', label='gain')
  plt.plot(x, -loss, c='c', label='loss')
  plt.legend(loc="upper left")

  plt.subplot(1, 3, 3)
  plt.title("expected utility")
  plt.plot(x, utility, c='y', label='utility')
  plt.legend(loc="upper left")

  plt.show()


def plot_utility_gaussian(mu1, mu2, sigma1, sigma2, mu_g, mu_c, sigma_g, sigma_c, plot_utility_row=True):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)

  mu_post, sigma_post = product_guassian(mu1, mu2, sigma1, sigma2)
  posterior = gaussian(x, mu_post, sigma_post)

  if plot_utility_row:
    gain = gaussian(x, mu_g, sigma_g)
    loss = gaussian(x, mu_c, sigma_c)
    utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)
    plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility)
  else:
    plot_bayes_row(x, prior, likelihood, posterior)

  return None


def plot_utility_mixture(mu_m1, mu_m2, sigma_m1, sigma_m2, factor,
                         mu, sigma, mu_g, mu_c, sigma_g, sigma_c, plot_utility_row=True):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  y_1 = gaussian(x, mu_m1, sigma_m1)
  y_2 = gaussian(x, mu_m2, sigma_m2)
  prior = y_1 * factor + y_2 * (1.0 - factor)

  likelihood = gaussian(x, mu, sigma)

  posterior = np.multiply(prior, likelihood)
  posterior = posterior / (posterior.sum() * (x[1] - x[0]))

  if plot_utility_row:
    gain = gaussian(x, mu_g, sigma_g)
    loss = gaussian(x, mu_c, sigma_c)
    utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)
    plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility)
  else:
    plot_bayes_row(x, prior, likelihood, posterior)

  return None


def plot_utility_uniform(mu, sigma, mu_g, mu_c, sigma_g, sigma_c, plot_utility_row=True):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  prior = np.ones_like(x) / (x.max() - x.min())
  likelihood = gaussian(x, mu, sigma)
  posterior = likelihood
  if plot_utility_row:
    gain = gaussian(x, mu_g, sigma_g)
    loss = gaussian(x, mu_c, sigma_c)
    utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)
    plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility)
  else:
    plot_bayes_row(x, prior, likelihood, posterior)

  return None


def plot_utility_gamma(alpha, beta, offset, mu, sigma, mu_g, mu_c, sigma_g, sigma_c, plot_utility_row=True):
  x = np.linspace(-4, 10, 1000, endpoint=True)
  prior = gamma_pdf(x-offset, alpha, beta)
  likelihood = gaussian(x, mu, sigma)

  posterior = np.multiply(prior, likelihood)
  posterior = posterior / (posterior.sum() * (x[1] - x[0]))

  if plot_utility_row:
    gain = gaussian(x, mu_g, sigma_g)
    loss = gaussian(x, mu_c, sigma_c)
    utility = np.multiply(posterior, gain) - np.multiply(posterior, loss)
    plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility)
  else:
    plot_bayes_row(x, prior, likelihood, posterior)

  return None


def plot_bayes_row(x, prior, likelihood, posterior):

  mean, median, mode = calc_mean_mode_median(x, posterior)

  plt.figure(figsize=(12, 4))
  plt.subplot(1, 2, 1)
  plt.title("Prior and likelihood distribution")
  plt.plot(x, prior, c='b', label='prior')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='likelihood')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.xlabel('x')

  plt.subplot(1, 2, 2)
  plt.title("Posterior distribution")
  plt.plot(x, posterior, c='k', label='posterior')
  plt.fill_between(x, posterior, color='k', alpha=0.1)
  plt.axvline(mean, ls='dashed', color='red', label='Mean')
  plt.axvline(median, ls='dashdot', color='blue', label='Median')
  plt.axvline(mode, ls='dotted', color='green', label='Mode')
  plt.legend(loc="upper left")
  plt.yticks([])
  plt.xlabel('x')

  plt.show()


def plot_bayes_utility_rows(x, prior, likelihood, posterior, gain, loss, utility):

  mean, median, mode = calc_mean_mode_median(x, posterior)
  max_utility = x[np.argmax(utility)]

  plt.figure(figsize=(12, 8))
  plt.subplot(2, 2, 1)
  plt.title("Prior and likelihood distribution")
  plt.plot(x, prior, c='b', label='prior')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='likelihood')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  # plt.plot(x, posterior, c='k', label='posterior')
  # plt.fill_between(x, posterior, color='k', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  # plt.ylabel('$f(x)$')
  plt.xlabel('x')

  plt.subplot(2, 2, 2)
  plt.title("Posterior distribution")
  plt.plot(x, posterior, c='k', label='posterior')
  plt.fill_between(x, posterior, color='k', alpha=0.1)
  plt.axvline(mean, ls='dashed', color='red', label='Mean')
  plt.axvline(median, ls='dashdot', color='blue', label='Median')
  plt.axvline(mode, ls='dotted', color='green', label='Mode')
  plt.legend(loc="upper left")
  plt.yticks([])
  plt.xlabel('x')

  plt.subplot(2, 2, 3)
  plt.title("utility function")
  plt.plot(x, gain, c='m', label='gain')
  # plt.fill_between(x, gain, color='m', alpha=0.2)
  plt.plot(x, -loss, c='c', label='loss')
  # plt.fill_between(x, -loss, color='c', alpha=0.2)
  plt.legend(loc="upper left")
  plt.xlabel('x')

  plt.subplot(2, 2, 4)
  plt.title("expected utility")
  plt.plot(x, utility, c='y', label='utility')
  # plt.fill_between(x, utility, color='y', alpha=0.2)
  plt.axvline(max_utility, ls='dashed', color='red', label='Max utility')
  plt.legend(loc="upper left")
  plt.xlabel('x')
  plt.ylabel('utility')
  plt.legend(loc="lower right")

  plt.show()


def plot_bayes_loss_utility_gaussian(loss_f, mu_true, mu1, mu2, sigma1, sigma2):
  x = np.linspace(-7, 7, 1000, endpoint=True)

  prior = gaussian(x, mu1, sigma1)
  likelihood = gaussian(x, mu2, sigma2)
  mu_post, sigma_post = product_guassian(mu1, mu2, sigma1, sigma2)
  posterior = gaussian(x, mu_post, sigma_post)

  loss = calc_loss_func(loss_f, mu_true, x)

  plot_bayes_loss_utility(x, prior, likelihood, posterior, loss, loss_f)

  return None


def plot_bayes_loss_utility_uniform(loss_f, mu_true, mu, sigma):
  x = np.linspace(-7, 7, 1000, endpoint=True)

  prior = np.ones_like(x) / (x.max() - x.min())
  likelihood = gaussian(x, mu, sigma)
  posterior = likelihood

  loss = calc_loss_func(loss_f, mu_true, x)

  plot_bayes_loss_utility(x, prior, likelihood, posterior, loss, loss_f)

  return None


def plot_bayes_loss_utility_gamma(loss_f, mu_true, alpha, beta, offset, mu, sigma):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  prior = gamma_pdf(x-offset, alpha, beta)
  likelihood = gaussian(x, mu, sigma)
  posterior = np.multiply(prior, likelihood)
  posterior = posterior / (posterior.sum() * (x[1] - x[0]))

  loss = calc_loss_func(loss_f, mu_true, x)

  plot_bayes_loss_utility(x, prior, likelihood, posterior, loss, loss_f)

  return None


def plot_bayes_loss_utility_mixture(loss_f, mu_true, mu_m1, mu_m2, sigma_m1, sigma_m2, factor, mu, sigma):
  x = np.linspace(-7, 7, 1000, endpoint=True)
  y_1 = gaussian(x, mu_m1, sigma_m1)
  y_2 = gaussian(x, mu_m2, sigma_m2)
  prior = y_1 * factor + y_2 * (1.0 - factor)
  likelihood = gaussian(x, mu, sigma)

  posterior = np.multiply(prior, likelihood)
  posterior = posterior / (posterior.sum() * (x[1] - x[0]))

  loss = calc_loss_func(loss_f, mu_true, x)

  plot_bayes_loss_utility(x, prior, likelihood, posterior, loss, loss_f)

  return None


def plot_bayes_loss_utility(x, prior, likelihood, posterior, loss, loss_f):

  mean, median, mode = calc_mean_mode_median(x, posterior)
  expected_loss = calc_expected_loss(loss_f, posterior, x)
  min_expected_loss = x[np.argmin(expected_loss)]

  plt.figure(figsize=(12, 8))

  plt.subplot(2, 2, 1)
  plt.title("Prior and Likelihood")
  plt.plot(x, prior, c='b', label='prior')
  plt.fill_between(x, prior, color='b', alpha=0.2)
  plt.plot(x, likelihood, c='r', label='likelihood')
  plt.fill_between(x, likelihood, color='r', alpha=0.2)
  plt.yticks([])
  plt.legend(loc="upper left")
  plt.xlabel('x')

  plt.subplot(2, 2, 2)
  plt.title("Posterior")
  plt.plot(x, posterior, c='k', label='posterior')
  plt.fill_between(x, posterior, color='k', alpha=0.1)
  plt.axvline(mean, ls='dashed', color='red', label='Mean')
  plt.axvline(median, ls='dashdot', color='blue', label='Median')
  plt.axvline(mode, ls='dotted', color='green', label='Mode')
  plt.legend(loc="upper left")
  plt.yticks([])
  plt.xlabel('x')

  plt.subplot(2, 2, 3)
  plt.title(loss_f)
  plt.plot(x, loss, c='c', label=loss_f)
  # plt.fill_between(x, loss, color='c', alpha=0.2)
  plt.ylabel('loss')
  plt.xlabel('x')

  plt.subplot(2, 2, 4)
  plt.title("expected loss")
  plt.plot(x, expected_loss, c='y', label='$\mathbb{E}[L]$')
  # plt.fill_between(x, expected_loss, color='y', alpha=0.2)
  plt.axvline(min_expected_loss, ls='dashed', color='red', label='$Min~ \mathbb{E}[Loss]$')
  plt.legend(loc="lower right")
  plt.xlabel('x')
  plt.ylabel('$\mathbb{E}[L]$')

  plt.show()


global global_loss_plot_switcher
global_loss_plot_switcher = False
def loss_plot_switcher(what_to_plot):
  global global_loss_plot_switcher
  if global_loss_plot_switcher:
    clear_output()
  else:
    global_loss_plot_switcher = True
  if what_to_plot == "Gaussian":
    loss_f_options = Dropdown(
                options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
                value="Mean Squared Error", description="Loss: ")
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_estimate", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_estimate", continuous_update=True)
    mu_true_slider = FloatSlider(min=-3.0, max=3.0, step=0.01, value=0.0, description="µ_true", continuous_update=True)

    widget_ui = HBox([VBox([loss_f_options, mu_true_slider]), VBox([mu_slider, sigma_slider])])

    widget_out = interactive_output(plot_loss_utility_gaussian,
                                    {'loss_f': loss_f_options,
                                    'mu': mu_slider,
                                    'sigma': sigma_slider,
                                    'mu_true': mu_true_slider})

  elif what_to_plot == "Mixture of Gaussians":
    loss_f_options = Dropdown(
            options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
            value="Mean Squared Error", description="Loss: ")

    mu1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_est_p", continuous_update=True)
    mu2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_est_q", continuous_update=True)
    sigma1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_est_p", continuous_update=True)
    sigma2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_est_q", continuous_update=True)
    factor_slider = FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="π", continuous_update=True)
    mu_true_slider = FloatSlider(min=-3.0, max=3.0, step=0.01, value=0.0, description="µ_true", continuous_update=True)

    widget_ui = HBox([VBox([loss_f_options, factor_slider, mu_true_slider]),
                      VBox([mu1_slider, sigma1_slider]),
                      VBox([mu2_slider, sigma2_slider])])

    widget_out = interactive_output(plot_loss_utility_mixture,
                                    {'mu1': mu1_slider,
                                    'mu2': mu2_slider,
                                    'sigma1': sigma1_slider,
                                    'sigma2': sigma2_slider,
                                    'factor': factor_slider,
                                    'mu_true': mu_true_slider,
                                    'loss_f': loss_f_options})
  display(widget_ui, widget_out)


global global_plot_prior_switcher
global_plot_prior_switcher = False
def plot_prior_switcher(what_to_plot):
  global global_plot_prior_switcher
  if global_plot_prior_switcher:
    clear_output()
  else:
    global_plot_prior_switcher = True

  if what_to_plot == "Gaussian":
    mu1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_prior", continuous_update=True)
    mu2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_prior", continuous_update=True)
    sigma2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)

    widget_ui = HBox([VBox([mu1_slider, sigma1_slider]),
                VBox([mu2_slider, sigma2_slider])])

    widget_out = interactive_output(plot_utility_gaussian,
                                    {'mu1': mu1_slider,
                                      'mu2': mu2_slider,
                                      'sigma1': sigma1_slider,
                                      'sigma2': sigma2_slider,
                                      'mu_g': fixed(1.0),
                                      'mu_c': fixed(-1.0),
                                      'sigma_g': fixed(0.5),
                                      'sigma_c': fixed(value=0.5),
                                      'plot_utility_row': fixed(False)})

  elif what_to_plot == "Uniform":
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)

    widget_ui = VBox([mu_slider, sigma_slider])

    widget_out = interactive_output(plot_utility_uniform,
                                    {'mu': mu_slider,
                                      'sigma': sigma_slider,
                                      'mu_g': fixed(1.0),
                                      'mu_c': fixed(-1.0),
                                      'sigma_g': fixed(0.5),
                                      'sigma_c': fixed(value=0.5),
                                      'plot_utility_row': fixed(False)})

  elif what_to_plot == "Gamma":
    alpha_slider = FloatSlider(min=1.0, max=10.0, step=0.1, value=2.0, description="α_prior", continuous_update=True)
    beta_slider = FloatSlider(min=0.5, max=2.0, step=0.01, value=1.0, description="β_prior", continuous_update=True)
    # offset_slider = FloatSlider(min=-6.0, max=2.0, step=0.1, value=0.0, description="offset", continuous_update=True)
    offset_slider = fixed(0.0)
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)
    gaus_label = Label(value="normal likelihood", layout=Layout(display="flex", justify_content="center"))
    gamma_label = Label(value="gamma prior", layout=Layout(display="flex", justify_content="center"))
    widget_ui = HBox([VBox([gamma_label, alpha_slider, beta_slider]),
                      VBox([gaus_label, mu_slider, sigma_slider])])

    widget_out = interactive_output(plot_utility_gamma,
                                    {'alpha': alpha_slider,
                                      'beta': beta_slider,
                                      'offset': offset_slider,
                                      'mu': mu_slider,
                                      'sigma': sigma_slider,
                                      'mu_g': fixed(1.0),
                                      'mu_c': fixed(-1.0),
                                      'sigma_g': fixed(0.5),
                                      'sigma_c': fixed(value=0.5),
                                      'plot_utility_row': fixed(False)})

  elif what_to_plot == "Mixture of Gaussians":
    mu_m1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_mix_p", continuous_update=True)
    mu_m2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_mix_q", continuous_update=True)
    sigma_m1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_mix_p", continuous_update=True)
    sigma_m2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_mix_q", continuous_update=True)
    factor_slider = FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="π", continuous_update=True)
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)

    widget_ui = HBox([VBox([mu_m1_slider, sigma_m1_slider, factor_slider]),
                      VBox([mu_m2_slider, sigma_m2_slider]),
                      VBox([mu_slider, sigma_slider])])

    widget_out = interactive_output(plot_utility_mixture,
                                    {'mu_m1': mu_m1_slider,
                                      'mu_m2': mu_m2_slider,
                                      'sigma_m1': sigma_m1_slider,
                                      'sigma_m2': sigma_m2_slider,
                                      'factor': factor_slider,
                                      'mu': mu_slider,
                                      'sigma': sigma_slider,
                                      'mu_g': fixed(1.0),
                                      'mu_c': fixed(-1.0),
                                      'sigma_g': fixed(0.5),
                                      'sigma_c': fixed(value=0.5),
                                      'plot_utility_row': fixed(False)})
  display(widget_ui, widget_out)


global global_plot_bayes_loss_utility_switcher
global_plot_bayes_loss_utility_switcher = False
def plot_bayes_loss_utility_switcher(what_to_plot):
  global global_plot_bayes_loss_utility_switcher
  if global_plot_bayes_loss_utility_switcher:
      clear_output()
  else:
      global_plot_bayes_loss_utility_switcher = True

  if what_to_plot == "Gaussian":
    loss_f_options = Dropdown(
                  options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
                  value="Mean Squared Error", description="Loss: ")
    mu1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_prior", continuous_update=True)
    sigma1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_prior", continuous_update=True)
    mu2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)
    mu_true_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_true", continuous_update=True)

    widget_ui = HBox([VBox([loss_f_options, mu1_slider, sigma1_slider]),
                      VBox([mu_true_slider, mu2_slider, sigma2_slider])])

    widget_out = interactive_output(plot_bayes_loss_utility_gaussian,
                                            {'mu1': mu1_slider,
                                            'mu2': mu2_slider,
                                            'sigma1': sigma1_slider,
                                            'sigma2': sigma2_slider,
                                            'mu_true': mu_true_slider,
                                            'loss_f': loss_f_options})

  elif what_to_plot == "Uniform":
    loss_f_options = Dropdown(
                  options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
                  value="Mean Squared Error", description="Loss: ")
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)
    mu_true_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_true", continuous_update=True)

    widget_ui = HBox([VBox([loss_f_options, mu_slider, sigma_slider]),
                      VBox([mu_true_slider])])

    widget_out = interactive_output(plot_bayes_loss_utility_uniform,
                                    {'mu': mu_slider,
                                    'sigma': sigma_slider,
                                    'mu_true': mu_true_slider,
                                    'loss_f': loss_f_options})

  elif what_to_plot == "Gamma":

    loss_f_options = Dropdown(
                  options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
                  value="Mean Squared Error", description="Loss: ")

    alpha_slider = FloatSlider(min=1.0, max=10.0, step=0.1, value=2.0, description="α_prior", continuous_update=True)
    beta_slider = FloatSlider(min=0.5, max=2.0, step=0.01, value=1.0, description="β_prior", continuous_update=True)
    # offset_slider = FloatSlider(min=-6.0, max=2.0, step=0.1, value=0.0, description="offset", continuous_update=True)
    offset_slider = fixed(0.0)
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)
    mu_true_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_true", continuous_update=True)
    gaus_label = Label(value="normal likelihood", layout=Layout(display="flex", justify_content="center"))
    gamma_label = Label(value="gamma prior", layout=Layout(display="flex", justify_content="center"))
    widget_ui = HBox([VBox([loss_f_options, gamma_label, alpha_slider, beta_slider]),
                      VBox([mu_true_slider, gaus_label, mu_slider, sigma_slider])])

    widget_out = interactive_output(plot_bayes_loss_utility_gamma,
                                    {'alpha': alpha_slider,
                                      'beta': beta_slider,
                                      'offset': offset_slider,
                                      'mu': mu_slider,
                                      'sigma': sigma_slider,
                                      'mu_true': mu_true_slider,
                                      'loss_f': loss_f_options})

  elif what_to_plot == "Mixture of Gaussians":
    mu_m1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_mix_p", continuous_update=True)
    mu_m2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_mix_q", continuous_update=True)
    sigma_m1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_mix_p", continuous_update=True)
    sigma_m2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_mix_q", continuous_update=True)
    factor_slider = FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="π", continuous_update=True)
    mu_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5, description="µ_likelihood", continuous_update=True)
    sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description="σ_likelihood", continuous_update=True)
    mu_true_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5, description="µ_true", continuous_update=True)
    loss_f_options = Dropdown(
        options=["Mean Squared Error", "Absolute Error", "Zero-One Loss"],
        value="Mean Squared Error", description="Loss: ")
    empty_label = Label(value=" ")

    widget_ui = HBox([VBox([loss_f_options, mu_m1_slider, sigma_m1_slider]),
                      VBox([mu_true_slider, mu_m2_slider, sigma_m2_slider]),
                      VBox([empty_label, mu_slider, sigma_slider])])

    widget_out = interactive_output(plot_bayes_loss_utility_mixture,
                                    {'mu_m1': mu_m1_slider,
                                      'mu_m2': mu_m2_slider,
                                      'sigma_m1': sigma_m1_slider,
                                      'sigma_m2': sigma_m2_slider,
                                      'factor': factor_slider,
                                      'mu': mu_slider,
                                      'sigma': sigma_slider,
                                      'mu_true': mu_true_slider,
                                      'loss_f': loss_f_options})
  display(widget_ui, widget_out)

## Funciones auxiliares

In [5]:
# @title Helper Functions

def gaussian(x, μ, σ):
    """ Compute Gaussian probability density function for given value of the
    random variable, mean, and standard deviation

    Args:
      x (scalar): value of random variable
      μ (scalar): mean of Gaussian
      σ (scalar): standard deviation of Gaussian

    Returns:
      scalar: value of probability density function
    """
    return np.exp(-((x - μ) / σ)**2 / 2) / np.sqrt(2 * np.pi * σ**2)


def gamma_pdf(x, α, β):
    return gamma_distribution.pdf(x, a=α, scale=1/β)


def mvn2d(x, y, mu1, mu2, sigma1, sigma2, cov12):
    mvn = multivariate_normal([mu1, mu2], [[sigma1**2, cov12], [cov12, sigma2**2]])
    return mvn.pdf(np.dstack((x, y)))


def product_guassian(mu1, mu2, sigma1, sigma2):
    J_1, J_2 = 1/sigma1**2, 1/sigma2**2
    J_3 = J_1 + J_2
    mu_prod = (J_1*mu1/J_3) + (J_2*mu2/J_3)
    sigma_prod = np.sqrt(1/J_3)
    return mu_prod, sigma_prod


def reverse_product(mu3, sigma3, mu1, mu2):
    J_3 = 1/sigma3**2
    J_1 = J_3 * (mu3 - mu2) / (mu1 - mu2)
    J_2 = J_3 * (mu3 - mu1) / (mu2 - mu1)
    sigma1, sigma2 = 1/np.sqrt(J_1), 1/np.sqrt(J_2)
    return sigma1, sigma2


def calc_mean_mode_median(x, y):
    """

    """
    pdf = y * (x[1] - x[0])
    # Calc mode of an arbitrary function
    mode = x[np.argmax(pdf)]

    # Calc mean of an arbitrary function
    mean = np.multiply(x, pdf).sum()

    # Calc median of an arbitrary function
    cdf = np.cumsum(pdf)
    idx = np.argmin(np.abs(cdf - 0.5))
    median = x[idx]

    return mean, median, mode


def calc_expected_loss(loss_f, posterior, x):
    dx = x[1] - x[0]
    expected_loss = np.zeros_like(x)
    for i in np.arange(x.shape[0]):
        loss = calc_loss_func(loss_f, x[i], x) # or mse or zero_one_loss
        expected_loss[i] = np.sum(loss * posterior) * dx
    return expected_loss

---
# Section 0: Introducción

---
# Sección 1: Astrogato!

<img src="https://github.com/NeuromatchAcademy/course-content/blob/main/tutorials/static/astrocat.png?raw=True">

Digamos que eres un gato astronauta: ¡Astrogato! Estás navegando por el espacio usando un jetpack y tu objetivo es perseguir un ratón.

Como eres un gato, no tienes pulgares oponibles y no puedes controlar tu propio jet pack. Sólo puede ser controlado mediante control terrestre en la Tierra.

Para que puedan guiarte, necesitan saber dónde estás. Están intentando averiguar su ubicación. Tienen conocimiento previo de tu ubicación: saben que te gusta pasar el rato cerca del ratón espacial. También pueden obtener un vistazo rápido y poco fiable: están tomando una medición del estado oculto de su ubicación.

Intentarán determinar su ubicación utilizando la regla de Bayes y las decisiones bayesianas, como veremos a lo largo de este tutorial.

---
# Sección 2: Distribución de probabilidad de la ubicación de Astrogato

Vamos a pensar primero en cómo control terrestre debería estimar su posición. No consideraremos mediciones todavía, sólo cómo representar la incertidumbre que podríamos tener en general. Ahora estamos tratando con una distribución continua: la ubicación de Astrogato puede ser cualquier número real. En el último tutorial, nos ocupamos de una distribución discreta: los peces estaban en un lado o en el otro.

Entonces, ¿cómo representamos la probabilidad de cada punto posible (un número infinito) donde podría estar el Astrogato?
El enfoque bayesiano se puede utilizar en cualquier distribución de probabilidad. Si bien muchas variables en el mundo requieren representación mediante distribuciones complejas o desconocidas (por ejemplo, empíricas), usaremos distribuciones gaussianas o extensiones de las mismas.

## Section 2.1: La distribución gaussiana



###  Video 3: La distribución gaussiana


La ecuación para una distribución gaussiana en una variable $x$ es:

\begin{equation}
\mathcal{N}(\mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(\frac{-(x-\mu)^2}{2\sigma^2}\right)
\end{equation}

En nuestro ejemplo, $x$ es la ubicación del Astrocat en una dirección. $\mathcal{N}(\mu,\sigma^2)$ es una notación estándar para referirse a una distribución **N**ormal (gaussiana). Por ejemplo, $\mathcal{N}(0, 1)$ denota una distribución gaussiana con media 0 y varianza 1. La forma exacta de esta ecuación no es particularmente intuitiva, pero veremos cómo los valores de media y desviación estándar afectan la probabilidad. distribución.

No implementaremos una distribución gaussiana en el código aquí, pero consulte la actualización de requisitos previos [W0D5 T1](https://precourse.neuromatch.io/tutorials/W0D5_Statistics/student/W0D5_Tutorial1.html) para hacer esto si necesita más aclaración.

In [6]:
# @markdown Ejecute esta celda para habilitar la función `gaussian`
def gaussian(x, mu, sigma):
  return np.exp(-(x - mu)**2 / (2*sigma**2)) / np.sqrt(2 * np.pi * sigma**2)

### Demo interactiva 2.1: Explorando los parámetros gaussianos

Exploremos cómo los parámetros de un gaussiano afectan la distribución. Juegue con la demostración a continuación cambiando la media $\mu$ y la desviación estándar $\sigma$.

Discutan lo siguiente:

1. ¿Qué hace el aumento de $\mu$? ¿Qué hace aumentar $\sigma$?
2. Si quisieras saber la probabilidad de que ocurra un evento en $0$, ¿puedes encontrar dos valores diferentes de $\mu$ y $\sigma$ que produzcan la misma probabilidad de un evento en $0$?
3. ¿Cuántas gaussianas podrían producir la misma probabilidad en $0$?

In [7]:
# @markdown Ejecute esta celda para habilitar el widget
widget = interact(plot_gaussian,
                  mu=FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.0,
                                 continuous_update=False, description="µ"),
                  sigma=FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5,
                                    continuous_update=False,description="σ"))

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='µ', max=4.0, min=-4.0, step…

## Sección 2.2: Multiplicando Gaussianas

### Interactive Demo 2.2: Multiplying Gaussians

We have implemented the multiplication of two Gaussians for you. Using the following widget, we are going to think about the information and combination of two Gaussians. In our case, imagine we want to find the middle location between the satellite and the space mouse. This would be the center (average) of the two locations. Because we have uncertainty, we need to weigh our uncertainty in thinking about the most likely place.

In this demo, $\mu_{1}$ and $\sigma_{1}$ are the mean and standard deviation of the distribution over satellite location, $\mu_{2}$ and $\sigma_{2}$ are the mean and standard deviation of the distribution over space mouse location, and $\mu_{3}$ and $\sigma_{3}$ are the mean and standard deviation of the distribution over the center location (gained by multiplying the first two).

Questions:

1. What is your uncertainty (how much information) do you have about $\mu_{3}$ with $\mu_{1} = -2, \mu_{2} = 2, \sigma_{1} = \sigma_{2} = 0.5$?
2. What happens to your estimate of $\mu_{3}$ as $\sigma_{2} \to \infty$? (In this case, $\sigma$ only goes to 11... but that should be loud enough.)
3. What is the difference in your estimate of $\mu_{3}$ if $\sigma_{1} = \sigma_{2} = 11$? What has changed from the first example?
4. Set $\mu_{1} = -4, \mu_{2} = 4$ and change the $\sigma$s so that $\mu_{3}$ is close to $2$. How many $\sigma$s will produce the same $\mu_{3}$?
5. Continuing, if you set $\mu_{1} = 0$, what $\sigma$ do you need to change so $\mu_{3} ~= 2$?
6. If $\sigma_{1} = \sigma_{2} = 0.1$, how much information do you have about the average?

 Execute this cell to enable the widget


In [8]:
# @markdown Execute this cell to enable the widget

mu1_slider = FloatSlider(min=-5.0, max=-0.51, step=0.01, value=-2.0,
                         description="µ_1", continuous_update=True)
mu2_slider = FloatSlider(min=0.5, max=5.01, step=0.01, value=2.0,
                         description="µ_2",continuous_update=True)
sigma1_slider = FloatSlider(min=0.1, max=11.01, step=0.01, value=1.0,
                            description="σ_1", continuous_update=True)
sigma2_slider = FloatSlider(min=0.1, max=11.01, step=0.01, value=1.0,
                            description="σ_2", continuous_update=True)
distro_1_label = Label(value="Satellite",
                       layout=Layout(display="flex", justify_content="center"))
distro_2_label = Label(value="Space Mouse",
                       layout=Layout(display="flex", justify_content="center"))
widget_ui = HBox([VBox([distro_1_label, mu1_slider, sigma1_slider]),
                  VBox([distro_2_label, mu2_slider, sigma2_slider])])

widget_out = interactive_output(plot_information, {'mu1': mu1_slider,
                                                   'mu2': mu2_slider,
                                                   'sigma1': sigma1_slider,
                                                   'sigma2': sigma2_slider})
display(widget_ui, widget_out)

Output()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/main/tutorials/W3D1_BayesianDecisions/solutions/W3D1_Tutorial2_Solution_0ee85a74.py)



####  Submit your feedback


In [9]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_Multiplying_Gaussians_Interactive_Demo_and_Discussion")

## Sección 2.3: Mezclas de Gaussianas

### Demo interactiva 2.3: Explorando mezclas gaussianas

Examinaremos una mezcla de dos gaussianos. Tendremos un parámetro de ponderación, π, que nos indica cómo ponderar uno de los gaussianos. El otro está ponderado por 1 - π.

Utilice el siguiente widget para experimentar con los parámetros de cada gaussiano y el peso de mezcla ($\pi$) para comprender cómo se comporta la mezcla de distribuciones gaussianas.

Discute las siguientes preguntas:

1. ¿Qué efecto tiene el aumento del peso $\pi$ en la distribución de la mezcla (azul oscuro)?
2. ¿Cómo se pueden separar más los dos picos de la distribución de la mezcla?
3. ¿Puedes hacer que la distribución de la mezcla tenga un solo pico  (como un gaussiano)?
4. ¿Alguna otra forma a la que puedas hacer que se asemeje la distribución de la mezcla además de un pico muy redondeado o dos picos separados?

In [10]:
# @markdown Execute this cell to enable the widget

mu1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-1.0,
                         description="µ_p", continuous_update=True)
mu2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=1.0,
                         description="µ_q", continuous_update=True)
sigma1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5,
                            description="σ_p", continuous_update=True)
sigma2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5,
                            description="σ_q", continuous_update=True)
factor_slider = FloatSlider(min=0.1, max=0.9, step=0.01, value=0.5,
                            description="π", continuous_update=True)
distro_1_label = Label(value="p(x)",
                       layout=Layout(display="flex", justify_content="center"))
distro_2_label = Label(value="q(x)",
                       layout=Layout(display="flex", justify_content="center"))
mixture_label = Label(value="mixing coefficient",
                      layout=Layout(display="flex", justify_content="center"))
widget_ui = HBox([VBox([distro_1_label, mu1_slider, sigma1_slider]),
                  VBox([distro_2_label, mu2_slider, sigma2_slider]),
                  VBox([mixture_label, factor_slider])])

widget_out = interactive_output(gaussian_mixture, {'mu1': mu1_slider,
                                                   'mu2': mu2_slider,
                                                   'sigma1': sigma1_slider,
                                                   'sigma2': sigma2_slider,
                                                   'factor': factor_slider})
display(widget_ui, widget_out)

Output()

In [11]:
# @markdown Execute this cell to enable the function `calc_loss_func`

def calc_loss_func(loss_f, mu_true, x):
    error = x - mu_true
    if loss_f == "Mean Squared Error":
        loss = (error)**2
    elif loss_f == "Absolute Error":
        loss = np.abs(error)
    elif loss_f == "Zero-One Loss":
        loss = (np.abs(error) >= 0.03).astype(np.float)
    return loss

---
# Sección 5: Teorema de Bayes para distribuciones continuas

## Sección 5.1: El ejemplo gaussiano

La regla de Bayes nos dice cómo combinar dos fuentes de información: el prior (por ejemplo, una representación ruidosa de las expectativas de Ground Control sobre dónde está Astrocat) y la verosimilitud (por ejemplo, una representación ruidosa de Astrocat después de tomar una medición), para obtener una distribución posterior (nuestra distribución de creencias) teniendo en cuenta ambas informaciones. Recuerde la regla de Bayes:

\begin{equation}
\text{Posterior} = \frac{ \text{Verosimilitud} \times \text{Prior}}{ \text{Constante de normalización}}
\end{equation}

Veremos qué sucede cuando tanto el a priori como la probabilidad son gaussianos. En estas ecuaciones, $\mathcal{N}(\mu,\sigma^2)$ denota una distribución gaussiana con parámetros $\mu$ y $\sigma^2$:

\begin{equation}
\mathcal{N}(\mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \; \exp \bigg( \frac{-(x-\mu)^2}{2\sigma^2} \bigg)
\end{equation}

Cuando tanto la prioridad como la probabilidad son gaussianas, la regla de Bayes se traduce de la siguiente forma:

\begin{align}
\text{Likelihood} &= \mathcal{N}(\mu_{likelihood},\sigma_{likelihood}^2) \\
\text{Prior} &= \mathcal{N}(\mu_{prior},\sigma_{prior}^2) \\
\text{Posterior} &= \mathcal{N}\left( \frac{\sigma^2_{likelihood}\mu_{prior}+\sigma^2_{prior}\mu_{likelihood}}{\sigma^2_{ likelihood}+\sigma^2_{prior}}, \frac{\sigma^2_{likelihood}\sigma^2_{prior}}{\sigma^2_{likelihood}+\sigma^2_{prior}} \right) \\
&\propto \mathcal{N}(\mu_{likelihood},\sigma_{likelihood}^2) \times \mathcal{N}(\mu_{prior},\sigma_{prior}^2)
\end{align}

Obtenemos los parámetros del posterior multiplicando las gaussianas, tal como lo hicimos en la sección 2.2.

### Demostración interactiva 5.1: Bayes gaussiano

In [12]:
# @markdown Execute this cell to enable the widget

mu1_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=-0.5,
                         description="µ_prior", continuous_update=True)
mu2_slider = FloatSlider(min=-4.0, max=4.0, step=0.01, value=0.5,
                         description="µ_likelihood", continuous_update=True)
sigma1_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5,
                            description="σ_prior", continuous_update=True)
sigma2_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5,
                            description="σ_likelihood", continuous_update=True)
distro1_label = Label(value="prior distribution",
                      layout=Layout(display="flex", justify_content="center"))
distro2_label = Label(value="likelihood distribution",
                      layout=Layout(display="flex", justify_content="center"))

widget_ui = HBox([VBox([distro1_label, mu1_slider, sigma1_slider]),
                  VBox([distro2_label, mu2_slider, sigma2_slider])])

widget_out = interactive_output(plot_bayes, {'mu1': mu1_slider,
                                             'mu2': mu2_slider,
                                             'sigma1': sigma1_slider,
                                             'sigma2': sigma2_slider})
display(widget_ui, widget_out)

Output()

### Pregunta: cuando le damos más pelota al prior y cuando a la verosimilitud?

## Sección 5.2: Explorando priors

### Demostración interactiva 5.2: Exploración del prior

¿Qué pasaría si tuviéramos un prior diferente para la ubicación de Astrocat? La regla de Bayes funciona exactamente de la misma manera si nuestro prior no es gaussiano (aunque la solución analítica puede ser mucho más compleja o imposible). Veamos cómo se comporta la parte posterior si tenemos un anterior diferente a la ubicación de Astrocat.

Considere las siguientes preguntas:

1. ¿Qué significa tener un prior chato?
2. ¿En qué se diferencia el prior gamma de los demás?

In [13]:
# @markdown Execute this cell to enable the widget
widget = interact(plot_prior_switcher,
                  what_to_plot = Dropdown(
                      options=["Gaussian", "Mixture of Gaussians",
                               "Uniform", "Gamma"],
                      value="Gaussian", description="Prior: "))

interactive(children=(Dropdown(description='Prior: ', options=('Gaussian', 'Mixture of Gaussians', 'Uniform', …